In [2]:
#installing packages in JupyterLab temporarily
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install bs4
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install datetime
!{sys.executable} -m pip install selenium
!{sys.executable} -m pip install tqdm
!{sys.executable} -m pip install ipywidgets
!{sys.executable} -m pip install jupyterlab_widgets

  Using cached pandas-1.2.4.tar.gz (5.5 MB)
  Installing build dependencies ... error
  ERROR: Command errored out with exit status 1:
   command: /Users/mateuszmajak/.local/share/virtualenvs/MotoScraper-EtnBywlc/bin/python /Users/mateuszmajak/.local/share/virtualenvs/MotoScraper-EtnBywlc/lib/python3.8/site-packages/pip install --ignore-installed --no-user --prefix /private/var/folders/09/8bg983y9483cw6gs7y74_1vw0000gn/T/pip-build-env-rdgy298w/overlay --no-warn-script-location --no-binary :none: --only-binary :none: -i https://pypi.org/simple -- setuptools wheel 'Cython>=0.29.21,<3' 'numpy==1.16.5; python_version=='"'"'3.7'"'"' and platform_system!='"'"'AIX'"'"'' 'numpy==1.17.3; python_version=='"'"'3.8'"'"' and platform_system!='"'"'AIX'"'"'' 'numpy==1.16.5; python_version=='"'"'3.7'"'"' and platform_system=='"'"'AIX'"'"'' 'numpy==1.17.3; python_version=='"'"'3.8'"'"' and platform_system=='"'"'AIX'"'"'' 'numpy; python_version>='"'"'3.9'"'"''
       cwd: None
  Complete output (4838 li

**Packages**

In [12]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as BS
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from datetime import datetime
import pandas as pd
import datetime
import getpass
import time
import re
from tqdm.notebook import trange, tqdm

**Brand categories select**

In [5]:
url = 'https://www.otomoto.pl/'

driver = webdriver.Firefox(executable_path=r'C:\Users\Admin\Downloads\geckodriver-v0.29.0-win64\geckodriver.exe')
driver.get(url)
# create BS object from the page source
bs = BS(driver.page_source, 'html.parser')

# waiting a few seconds to be sure that page will be fully loaded
time.sleep(3)
cookie_button = driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]').click()

#loop to find all of the brands
categories_links = []

for i in tqdm(range(147)):
    time.sleep(1)
    brand_button = driver.find_element_by_xpath('//*[@id="__next"]/div/div/div/main/div[2]/article/div/form/div[1]/div[1]/div/div/div').click()
    brand_dropdown_button = driver.find_element_by_xpath(('//*[@id="downshift-0-item-{}"]/div/span').format(i+9)).click() #items 9-155
    #brand_dropdown_button = driver.find_element_by_xpath('//*[@id="downshift-0-item-9"]/div/span').click() #items 9-155
    time.sleep(3)
    enter = driver.find_element_by_xpath('//*[@id="filter_enum_make"]').send_keys(Keys.RETURN)
    time.sleep(3)
    categories_links.append(driver.current_url)
    driver.get(url)

driver.quit()

  0%|          | 0/147 [00:00<?, ?it/s]

In [13]:
with open('categories_links.txt') as f:
    categories_links = f.read().splitlines()

In [4]:
with open('categories_links.txt', 'w') as f:
    for item in categories_links:
        f.write("%s\n" % item)

NameError: name 'categories_links' is not defined

**All pages with urls scraping**

In [37]:
categories_links[25:35]

['https://www.otomoto.pl/osobowe/chevrolet/',
 'https://www.otomoto.pl/osobowe/chrysler/',
 'https://www.otomoto.pl/osobowe/citroen/',
 'https://www.otomoto.pl/osobowe/comarth/',
 'https://www.otomoto.pl/osobowe/cupra/',
 'https://www.otomoto.pl/osobowe/dacia/',
 'https://www.otomoto.pl/osobowe/daewoo/',
 'https://www.otomoto.pl/osobowe/daihatsu/',
 'https://www.otomoto.pl/osobowe/de-lorean/',
 'https://www.otomoto.pl/osobowe/dfsk/']

In [43]:
############################################################################
# Downloading offers' urls and titles from the website controlled by Selenium
import os

#url = 'https://www.otomoto.pl/osobowe/'
options = webdriver.firefox.options.Options()
# options = webdriver.chrome.options.Options()

options.headless = False
driver = webdriver.Firefox(executable_path='/Users/mateuszmajak/Documents/gecko/geckodriver')

url_list = []
title_list = []
    
for url in tqdm(categories_links):
    driver.get(url)
    # create BS object from the page source
    bs = BS(driver.page_source, 'html.parser')
    #### click accept COOKIEs button
    # waiting a few seconds to be sure that page will be fully loaded
    time.sleep(4)
    try:
        button = driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]').click()
    except:
        button = ''
    # Extracting number of pages with offers list to iterate
    try:
        maxSites = (bs.find_all('span', {'class':'page'})[-1].text)
        print("Max site number: " + maxSites)
    except:
        maxSites = 0
    # All offers' titles and urls scraping

    for i in range(int(maxSites)):
        if(i > 0):
            try:
                button = driver.find_element_by_xpath('//span[@class="icon-arrow_right"]').click()
            except:
                button = ''
        time.sleep(3)
        bs = BS(driver.page_source, 'html.parser')
        titles = bs.find_all('a', {'class':'offer-title__link'}) 
        links = [title['href'] for title in titles]
        titles_text = [title.text.strip() for title in titles]
        for link in links:
            url_list.append(link)
        for title in titles_text:
            title_list.append(title)
offers = pd.DataFrame(list(zip(title_list, url_list)),
                      columns =['title', 'url'])
offers.to_csv('auction_urls_new.csv', index=False)
#     if not os.path.isfile('auction_urls.csv'):
#         offers.to_csv('auction_urls.csv', header='column_names', index=False)
#     else: # else it exists so append without writing the header
#         offers.to_csv('auction_urls.csv', mode='a', header=False, index=False)
    # Close browser:
driver.quit()

  0%|          | 0/147 [00:00<?, ?it/s]

Max site number: 2
Max site number: 4
Max site number: 43
Max site number: 500
Max site number: 3
Max site number: 500
Max site number: 2
Max site number: 5
Max site number: 48
Max site number: 22
Max site number: 232
Max site number: 5
Max site number: 52
Max site number: 4
Max site number: 5
Max site number: 22
Max site number: 4
Max site number: 2
Max site number: 161
Max site number: 500
Max site number: 129
Max site number: 184
Max site number: 9
Max site number: 2
Max site number: 3
Max site number: 31
Max site number: 56
Max site number: 186
Max site number: 2
Max site number: 9
Max site number: 40
Max site number: 32
Max site number: 2
Max site number: 3
Max site number: 6
Max site number: 144
Max site number: 370
Max site number: 2
Max site number: 44
Max site number: 61
Max site number: 152
Max site number: 500
Max site number: 282
Max site number: 2
Max site number: 36
Max site number: 2
Max site number: 339
Max site number: 2
Max site number: 15
Max site number: 154
Max sit

In [10]:
# print(title_list[1])
# print("#############")
# print(url_list[1])

['Abarth 124 1.4', 'Abarth 595', 'Abarth 595', 'Abarth Grande Punto', 'Abarth 500', 'Abarth 595', 'Abarth 595', 'Abarth 595', 'Abarth 595', 'Abarth 124 1.4', 'Abarth 595', 'Abarth 595', 'Abarth 500', 'Abarth 595', 'Abarth 595', 'Abarth 500', 'Abarth Grande Punto', 'Abarth 595', 'Abarth 595', 'Abarth 500', 'Abarth 595', 'Abarth 595', 'Abarth 595', 'Abarth 595', 'Abarth 595', 'Abarth 595', 'Abarth 595', 'Abarth 595', 'Abarth 595', 'Abarth 595', 'Abarth 500', 'Abarth 595', 'Abarth 595', 'Abarth 595', 'Abarth Grande Punto', 'Abarth 595', 'Abarth 500', 'Abarth 500 595', 'Abarth 595', 'Abarth 595', 'Abarth 500']
#############
https://www.otomoto.pl/oferta/abarth-595-lekko-uszkodzony-szklany-dach-piekny-ID6DLAcl.html#d0ce083b14


In [19]:
# Save urls to df
# urls_df = pd.DataFrame({'offer_title': title_list, 'link': url_list})
# urls_df.to_csv('urls.csv', index=False, encoding="Windows-1252")

In [44]:
# Read urls from csv 
urls_df = pd.read_csv('auction_urls.csv')
url_list = urls_df['url'].tolist()

In [283]:
###Single site test
from urllib import request
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as BS
import pandas as pd
import re
from datetime import datetime

headerOne = {'User-Agent': 'Mozilla/5.0'}

cars = pd.DataFrame({'offer_id':[], 'offer_date':[], 'brand':[], 'model':[], 'version':[], 'generation':[], 'seller':[], 'year_production':[], 
                      'mileage':[], 'engine_capacity':[], 'fuel_type':[], 'power':[], 'gearbox':[],
                     'drive_wheel':[], 'CO2_emission':[], 'vehicle_type':[], 'door_number':[], 'seats_number':[],
                     'color':[], 'registered_in_poland':[], 'service_dealership':[], 'vehicle_condition':[], 'never_crashed':[],
                     'location':[], 'seller_registration':[], 'offer_price_details':[], 'offer_price':[],
      'offer_price_currency':[], 'equipment':[]})

site = 'https://www.otomoto.pl/oferta/volvo-xc-60-piekne-xc60-r-design-165-tys-km-serwis-orginal-xenon-skora-full-ID6DEw6n.html#7ce9f71ee3'

req = Request(site, headers=headerOne)

html = urlopen(req)
bs = BS(html.read(), 'html.parser')

try:
    brand = bs.find('span', text='Marka pojazdu').parent.a.text.strip()
except:
    brand = ''

# print('This is the brand: ', brand)

try:
    model = bs.find('span', text='Model pojazdu').parent.a.text.strip()
except:
    model = ''

# print('This is the model: ', model)

try:
    version = bs.find('span', text='Wersja').parent.a.text.strip()
except:
    version = ''

# print('This is the version: ', version)

try:
    generation = bs.find('span', text='Generacja').parent.a.text.strip()
except:
    generation = ''

# print('This is the generation: ', generation)

try:
    year_production = bs.find('span', text='Rok produkcji').parent.div.text.strip()
except:
    year_production = ''

# print('This is the year_production: ', year_production)

try:
    seller = bs.find('span', text='Oferta od').parent.a.text.strip()
except:
    seller = ''

# print('This is the seller: ', seller)

try:
    mileage = bs.find('span', text='Przebieg').parent.div.text.strip()
except:
    mileage = ''

# print('This is the mileage: ', mileage)

try:
    engine_capacity = bs.find('span', text='Pojemność skokowa').parent.div.text.strip()
except:
    engine_capacity = ''

# print('This is the engine_capacity: ', engine_capacity)

try:
    fuel_type = bs.find('span', text='Rodzaj paliwa').parent.a.text.strip()
except:
    fuel_type = ''

# print('This is the fuel_type: ', fuel_type)

try:
    power = bs.find('span', text='Moc').parent.div.text.strip()
except:
    power = ''

# print('This is the power: ', power)

try:
    gearbox = bs.find('span', text='Skrzynia biegów').parent.a.text.strip()
except:
    gearbox = ''

# print('This is the gearbox: ', gearbox)

try:
    drive_wheel = bs.find('span', text='Napęd').parent.a.text.strip()
except:
    drive_wheel = ''

# print('This is the drive_wheel: ', drive_wheel)

try:
    CO2_emission = bs.find('span', text='Emisja CO2').parent.div.text.strip()
except:
    CO2_emission = ''

# print('This is the CO2_emission: ', CO2_emission)

try:
    vehicle_type = bs.find('span', text='Typ').parent.a.text.strip()
except:
    vehicle_type = ''

# print('This is the vehicle_type: ', vehicle_type)

try:
    door_number = bs.find('span', text='Liczba drzwi').parent.div.text.strip()
except:
    door_number = ''

# print('This is the door_number: ', door_number)

try:
    seats_number = bs.find('span', text='Liczba miejsc').parent.div.text.strip()
except:
    seats_number = ''

# print('This is the seats_number: ', seats_number)

try:
    color = bs.find('span', text='Kolor').parent.a.text.strip()
except:
    color = ''

# print('This is the color: ', color)

try:
    registered_in_poland = bs.find('span', text='Zarejestrowany w Polsce').parent.a.text.strip()
except:
    registered_in_poland = ''

# print('This is the registered_in_poland: ', registered_in_poland)

try:
    never_crashed = bs.find('span', text='Bezwypadkowy').parent.a.text.strip()
except:
    never_crashed = ''

# print('This is the never_crashed: ', never_crashed)

try:
    service_dealership = bs.find('span', text='Serwisowany w ASO').parent.a.text.strip()
except:
    service_dealership = ''

# print('This is the service_dealership: ', service_dealership)

try:
    vehicle_condition = bs.find('span', text='Stan').parent.a.text.strip()
except:
    vehicle_condition = ''

# print('This is the vehicle_condition: ', vehicle_condition)

try:
    offer_date = bs.find('div', {'class':'offer-content__metabar'}).div.span.text.strip()
except:
    offer_date = ''

# print('This is the offer_date: ', offer_date)

try:
    offer_id = bs.find('span', {'id':'ad_id'}).text
except:
    offer_id = ''

# print('This is the offer_id: ', offer_id)

try:
    location = bs.find('span', {'class':'seller-box__seller-address__label'}).text.strip()
except:
    location = ''

# print('This is the location: ', location)

try:
    seller_registration = bs.find('div', {'class':'seller-box__seller-registration'}).text.strip()
    seller_registration = re.findall('[1-2][0-9][0-9][0-9]', seller_registration)[0]
except:
    seller_registration = ''

# print('This is the seller_registration: ', seller_registration)

try:
    offer_price_details = bs.find('span', {'class':'offer-price__details'}).text.strip()
except:
    offer_price_details = ''

# print('This is the offer_price_details: ', offer_price_details)

try:
    offer_price = bs.find('span', {'class':'offer-price__number'}).text.replace(" ", "")
    offer_price = re.sub('[^0-9]', "", offer_price)
except:
    offer_price = ''

# print('This is the offer_price: ', offer_price)

try:
    offer_price_currency = bs.find('span', {'class':'offer-price__currency'}).text.strip()
except:
    offer_price_currency = ''

# print('This is the offer_price_currency: ', offer_price_currency)

try:
    equipment = bs.findAll('li', {'class':'offer-features__item'})
    equipment = [a.text.strip() for a in equipment]
except:
    equipment = ''

# print('This is the equipment: ', equipment)

car = {}

car = {'offer_id':offer_id, 'offer_date':offer_date, 'brand':brand, 'model':model, 'version':version, 'generation':generation, 'seller':seller, 'year_production':year_production, 
       'mileage':mileage, 'engine_capacity':engine_capacity, 'fuel_type':fuel_type, 'power':power, 'gearbox':gearbox,
       'drive_wheel':drive_wheel, 'CO2_emission':CO2_emission, 'vehicle_type':vehicle_type, 'door_number':door_number, 'seats_number':seats_number,
       'color':color, 'registered_in_poland':registered_in_poland, 'service_dealership':service_dealership, 'vehicle_condition':vehicle_condition,
       'never_crashed':never_crashed, 'location':location, 'seller_registration':seller_registration, 'offer_price_details':offer_price_details,
       'offer_price':offer_price, 'offer_price_currency':offer_price_currency, 'equipment':equipment}

cars = cars.append(car, ignore_index = True)
print(cars)

     offer_id           offer_date  brand  model      version     generation  \
0  6082726775  23:36, 4 marca 2021  Volvo  XC 60  D3 R-Design  I (2008-2017)   

            seller year_production     mileage engine_capacity  ...  \
0  Osoby prywatnej            2011  165 000 km       1 984 cm3  ...   

  registered_in_poland service_dealership vehicle_condition never_crashed  \
0                  Tak                Tak           Używane           Tak   

                 location seller_registration offer_price_details offer_price  \
0  Kielce, Świętokrzyskie                2015       Do negocjacji       50500   

  offer_price_currency                                          equipment  
0                  PLN  [ABS, Elektryczne szyby przednie, Poduszka pow...  

[1 rows x 29 columns]


In [ ]:
# All pages scraper
headerOne = {'User-Agent': 'Mozilla/5.0'}

cars = pd.DataFrame({'offer_id':[], 'offer_date':[], 'brand':[], 'model':[], 'version':[], 'generation':[], 'seller':[], 'year_production':[], 
                      'mileage':[], 'engine_capacity':[], 'fuel_type':[], 'power':[], 'gearbox':[],
                     'drive_wheel':[], 'CO2_emission':[], 'vehicle_type':[], 'door_number':[], 'seats_number':[],
                     'color':[], 'registered_in_poland':[], 'service_dealership':[], 'vehicle_condition':[], 'never_crashed':[],
                     'location':[], 'seller_registration':[], 'offer_price_details':[], 'offer_price':[],
      'offer_price_currency':[], 'equipment':[]})

i = 0
for url in tqdm(url_list):
    site = url
    req = Request(site, headers=headerOne)
    try:
        html = urlopen(req)
        bs = BS(html.read(), 'html.parser')
    except:
        continue
    
    try:
        brand = bs.find('span', text='Marka pojazdu').parent.a.text.strip()
    except:
        brand = ''

    try:
        model = bs.find('span', text='Model pojazdu').parent.a.text.strip()
    except:
        model = ''

    try:
        version = bs.find('span', text='Wersja').parent.a.text.strip()
    except:
        version = ''

    try:
        generation = bs.find('span', text='Generacja').parent.a.text.strip()
    except:
        generation = ''

    try:
        year_production = bs.find('span', text='Rok produkcji').parent.div.text.strip()
    except:
        year_production = ''

    try:
        seller = bs.find('span', text='Oferta od').parent.a.text.strip()
    except:
        seller = ''

    try:
        mileage = bs.find('span', text='Przebieg').parent.div.text.strip()
    except:
        mileage = ''

    try:
        engine_capacity = bs.find('span', text='Pojemność skokowa').parent.div.text.strip()
    except:
        engine_capacity = ''

    try:
        fuel_type = bs.find('span', text='Rodzaj paliwa').parent.a.text.strip()
    except:
        fuel_type = ''

    try:
        power = bs.find('span', text='Moc').parent.div.text.strip()
    except:
        power = ''

    try:
        gearbox = bs.find('span', text='Skrzynia biegów').parent.a.text.strip()
    except:
        gearbox = ''

    try:
        drive_wheel = bs.find('span', text='Napęd').parent.a.text.strip()
    except:
        drive_wheel = ''

    try:
        CO2_emission = bs.find('span', text='Emisja CO2').parent.div.text.strip()
    except:
        CO2_emission = ''

    try:
        vehicle_type = bs.find('span', text='Typ').parent.a.text.strip()
    except:
        vehicle_type = ''

    try:
        door_number = bs.find('span', text='Liczba drzwi').parent.div.text.strip()
    except:
        door_number = ''

    try:
        seats_number = bs.find('span', text='Liczba miejsc').parent.div.text.strip()
    except:
        seats_number = ''

    try:
        color = bs.find('span', text='Kolor').parent.a.text.strip()
    except:
        color = ''

    try:
        registered_in_poland = bs.find('span', text='Zarejestrowany w Polsce').parent.a.text.strip()
    except:
        registered_in_poland = ''

    try:
        never_crashed = bs.find('span', text='Bezwypadkowy').parent.a.text.strip()
    except:
        never_crashed = ''

    try:
        service_dealership = bs.find('span', text='Serwisowany w ASO').parent.a.text.strip()
    except:
        service_dealership = ''

    try:
        vehicle_condition = bs.find('span', text='Stan').parent.a.text.strip()
    except:
        vehicle_condition = ''

    try:
        offer_date = bs.find('div', {'class':'offer-content__metabar'}).div.span.text.strip()
    except:
        offer_date = ''

    try:
        offer_id = bs.find('span', {'id':'ad_id'}).text
    except:
        offer_id = ''

    try:
        location = bs.find('span', {'class':'seller-box__seller-address__label'}).text.strip()
    except:
        location = ''

    try:
        seller_registration = bs.find('div', {'class':'seller-box__seller-registration'}).text.strip()
        seller_registration = re.findall('[1-2][0-9][0-9][0-9]', seller_registration)[0]
    except:
        seller_registration = ''

    try:
        offer_price_details = bs.find('span', {'class':'offer-price__details'}).text.strip()
    except:
        offer_price_details = ''

    try:
        offer_price = bs.find('span', {'class':'offer-price__number'}).text.replace(" ", "")
        offer_price = re.sub('[^0-9]', "", offer_price)
    except:
        offer_price = ''

    try:
        offer_price_currency = bs.find('span', {'class':'offer-price__currency'}).text.strip()
    except:
        offer_price_currency = ''

    try:
        equipment = bs.findAll('li', {'class':'offer-features__item'})
        equipment = [a.text.strip() for a in equipment]
    except:
        equipment = ''

    i += 1    
        
    car = {}

    car = {'offer_id':offer_id, 'offer_date':offer_date, 'brand':brand, 'model':model, 'version':version, 'generation':generation, 'seller':seller, 'year_production':year_production, 
       'mileage':mileage, 'engine_capacity':engine_capacity, 'fuel_type':fuel_type, 'power':power, 'gearbox':gearbox,
       'drive_wheel':drive_wheel, 'CO2_emission':CO2_emission, 'vehicle_type':vehicle_type, 'door_number':door_number, 'seats_number':seats_number,
       'color':color, 'registered_in_poland':registered_in_poland, 'service_dealership':service_dealership, 'vehicle_condition':vehicle_condition,
       'never_crashed':never_crashed, 'location':location, 'seller_registration':seller_registration, 'offer_price_details':offer_price_details,
       'offer_price':offer_price, 'offer_price_currency':offer_price_currency, 'equipment':equipment}

    cars = cars.append(car, ignore_index = True)
    
    cars.to_csv('auctions_final.csv', encoding="utf-8")

  0%|          | 0/203054 [00:00<?, ?it/s]